In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel('./4.auth.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：1


['40-10000']

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:

    while True:
        b += 1
        print(b)
        try:
            resp = requests.get(f'''https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:{input_.loc[a, 'Part Number']})&size=5000&useCache=true&auth={input_.loc[a, 'Auth']}''',
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))
            break
        except:
            continue

    if resp.status_code == 200:
        break

resp

1


<Response [200]>

In [6]:
import json

list_json = resp.json()

list_json

[]

In [7]:
list_make = [json_['fitment']['Vcdb']['MakeName'].strip() for json_ in list_json]

list_make

[]

In [8]:
list_model = [json_['fitment']['Vcdb']['ModelName'].strip() for json_ in list_json]

list_model

[]

In [9]:
list_region = [json_['fitment']['Vcdb']['RegionName'].strip() for json_ in list_json]

list_region

[]

In [10]:
list_submodel = [json_['fitment']['Vcdb']['SubModelName'].strip() for json_ in list_json]

list_submodel

[]

In [11]:
list_year = [int(json_['fitment']['Vcdb']['Year'].strip()) for json_ in list_json]

list_year

[]

In [12]:
list_list_aspiration, list_list_cylinder, list_list_displacement, list_list_fuel, list_list_liter = [], [], [], [], []
list_list_engine = [json_['fitment']['Vcdb']['Engines'] for json_ in list_json]
for list_engine in list_list_engine:
    if list_engine:
        list_aspiration, list_cylinder, list_displacement, list_fuel, list_liter = [], [], [], [], []
        for engine in list_engine:
            list_aspiration.append(engine['AspirationName'].strip() if 'AspirationName' in engine and engine['AspirationName'].strip() != '-' else '')

            # = = = = = = = = = = = = = = =

            cylinder = ''
            if 'BlockType' in engine and engine['BlockType'].strip() != '-':
                cylinder += engine['BlockType'].strip()
            if 'Cylinders' in engine and engine['Cylinders'] != 0:
                cylinder += str(engine['Cylinders'])
            list_cylinder.append(cylinder)

            # = = = = = = = = = = = = = = =

            displacement = ''
            if engine['CID'].strip() != '-':
                displacement += f'''{engine['CID'].strip()}cid'''
            elif engine['CC'].strip() != '-':
                displacement += f'''/{engine['CC'].strip()}cc'''
            list_displacement.append(displacement.removeprefix('/'))

            # = = = = = = = = = = = = = = =

            list_fuel.append(engine['FuelTypeName'].strip() if 'FuelTypeName' in engine else '')

            # = = = = = = = = = = = = = = =

            list_liter.append(f'''{engine['Liter']}L''' if 'Liter' in engine and engine['Liter'] != 0 else '')

        # = = = = = = = = = = = = = = =

        list_list_aspiration.append(list_aspiration)
        list_list_cylinder.append(list_cylinder)
        list_list_displacement.append(list_displacement)
        list_list_fuel.append(list_fuel)
        list_list_liter.append(list_liter)
    else:
        list_list_aspiration.append([''])
        list_list_cylinder.append([''])
        list_list_displacement.append([''])
        list_list_fuel.append([''])
        list_list_liter.append([''])

list_list_aspiration, list_list_cylinder, list_list_displacement, list_list_fuel, list_list_liter

([], [], [], [], [])

In [13]:
list_list_drive = []
list_list_drive_type = [json_['fitment']['Vcdb']['DriveTypes'] for json_ in list_json]
for list_drive_type in list_list_drive_type:
    if list_drive_type:
        list_drive = []
        for drive_type in list_drive_type:
            list_drive.append(drive_type['DriveTypeName'].strip())

        # = = = = = = = = = = = = = = =

        list_list_drive.append(list_drive)
    else:
        list_list_drive.append([''])

list_list_drive

[]

In [14]:
list_position = [json_['fitment']['acesCriteria']['PositionName'].strip() for json_ in list_json]

list_position

[]

In [15]:
df_temp = pd.DataFrame()
for i in range(len(list_json)):
    df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                 'Part Number': input_.loc[a, 'Part Number'],
                                 'Make': list_make[i],
                                 'Model': list_model[i],
                                 'Region': list_region[i],
                                 'Submodel': list_submodel[i],
                                 'Year': list_year[i],
                                 'Aspiration': list_list_aspiration[i],
                                 'Cylinder': list_list_cylinder[i],
                                 'Displacement': list_list_displacement[i],
                                 'Fuel': list_list_fuel[i],
                                 'Liter': list_list_liter[i]})
    for j in range(len(list_list_drive_type[i])):
        df_temp_temp_temp = df_temp_temp.copy()
        df_temp_temp_temp['Drive Type'] = list_list_drive[i][j]
        df_temp = pd.concat([df_temp, df_temp_temp_temp], ignore_index=True).fillna('')
    df_temp['Position'] = list_position[i]

if not df_temp.empty:
    df_temp['Row'] = [i+1 for i in range(len(df_temp))]
    df_temp = df_temp.drop_duplicates(['No', 'Part Number', 'Make', 'Model', 'Region', 'Submodel', 'Year', 'Aspiration', 'Cylinder', 'Displacement', 'Fuel', 'Liter', 'Drive Type', 'Position'],
                                      keep='first',
                                      ignore_index=True)
    df_temp = df_temp.sort_values(by=['No', 'Row'],
                                  ascending=[True, True],
                                  ignore_index=True)
    df_temp['Row'] = [i+1 for i in range(len(df_temp))]
    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

""


In [16]:
crawler_status = 'ok'

crawler_status

'ok'

In [17]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                             'Part Number': input_.loc[a, 'Part Number'],
                             'Url': input_.loc[a, 'Url'],
                             'Auth': input_.loc[a, 'Auth']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [18]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No', 'Row'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle_1.xlsx', index=False)

output_correct

""


In [19]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_1_error.xlsx', index=False)

output_error

""
